In [ ]:
# Sample code for 06-dashboard.ipynb
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Load data
df = pd.read_csv('../data/processed_data/falcon9_processed.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard"),
    
    html.Div([
        html.Div([
            html.H3("Launch Site Selection:"),
            dcc.Dropdown(
                id='site-dropdown',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                ],
                value='ALL',
                placeholder="Select a Launch Site"
            )
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.H3("Payload Range (kg):"),
            dcc.RangeSlider(
                id='payload-slider',
                min=0,
                max=10000,
                step=1000,
                value=[0, 10000],
                marks={i: f'{i}kg' for i in range(0, 10001, 2000)}
            )
        ], style={'width': '48%', 'display': 'inline-block'})
    ]),
    
    html.Div([
        dcc.Graph(id='success-pie-chart')
    ]),
    
    html.Div([
        dcc.Graph(id='success-payload-scatter-chart')
    ])
])

# Callback for pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(df, names='landing_success', 
                     title='Total Success vs Failure for All Sites',
                     color_discrete_map={0: 'red', 1: 'green'})
    else:
        filtered_df = df[df['launch_site'] == selected_site]
        fig = px.pie(filtered_df, names='landing_success', 
                     title=f'Success vs Failure for {selected_site}',
                     color_discrete_map={0: 'red', 1: 'green'})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)